In [4]:
from jsonobject import *
import numpy as np 
import random
import math
import networkx as nx
from networkx.readwrite import json_graph
import matplotlib as mpl
import matplotlib.pyplot as plt
from enum import Enum

In [5]:
random.seed(2021)

# Classes

In [6]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [7]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)

In [8]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="valid")
    stride = IntegerProperty(default=None)

In [9]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [10]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

In [11]:
# identity block 
class IdBlock(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    nb_filter = IntegerProperty()
    stride = 1
    fct_activation = StringProperty()

In [12]:
# convolution block
class ConvBlock(JsonObject):
    kernel = IntegerProperty()
    nb_filter = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    fct_activation = StringProperty()

In [13]:
# Enum type archi
class TypeArchi(Enum):
    ALL = 0
    LENET = 1
    RESNET = 2
    DENSENET = 3

In [14]:
# configuration Archi
class ConfArchi(JsonObject):
    type_archi = IntegerProperty()
    epsilon = FloatProperty()
    dropout_rate = FloatProperty()
    compress_factor = FloatProperty()
    

In [15]:
# DenseNet Convolution Block 
class DenseNetBlock(JsonObject):
    kernel = IntegerProperty()
    nb_filter = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_layer = IntegerProperty()
    fct_activation = StringProperty()
    op = StringProperty()
    nb_block_densenet = IntegerProperty()

In [16]:
# Global pooling 
class GlobalPooling(JsonObject):
    op = StringProperty()

# Value

In [17]:
# Table of hyperparameter value
kernel_value = [2, 3, 4, 5, 6, 7]
stride_value = [1, 2, 3, 4, 5, 6, 7]
padding_value = ["valid", "same"]
fct_activation_value = ["tanh", "relu", "selu"]

epsilon_value = [0, 1.001e-5, 0.001, 1.1e-5, 1.1e-7]
dropout_value = [0, .1, .4, .5, .8, .01, .001]

nb_layer_value = [1,2,3]
nb_block_densenet_value = [1,2,3,4,5]

nb_class = 10 # nb_class
op_value = ['avg',"max"]

classification = [Flatten, GlobalPooling]

# Functions

In [18]:
def stop(size_archi, x):
    prob = (math.log(2*size_archi)*x)-(2*x)
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [19]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    # open file
    archi_file = open(directory+file_name, "a")
    
    archi_file.write("""[
    """)
    
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [20]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        if(input_size%stride == 0):
            output_size = int(input_size/stride)
        else:
            output_size = int(input_size/stride)+1
    return output_size

In [21]:
# add layer to architecture 

def addLayer(archi, layer):
    input_size = archi[0].shape[0]
    feature_extra = archi[1:]
    
    # add layer if the architecture is empty
    if(feature_extra == []):
        archi.append(layer)
        return 1, archi, calcul_output(input_size,layer)
    
    else :
        # compute size of the output of the last layer
        for l in feature_extra: 
            output_size = calcul_output(input_size, l)
            input_size = output_size
        
        # if we couldn't reduce more
        if(input_size == 1):
            return 0, archi, input_size
        
        # if the output size got more than 1 we can add new layer
        elif(output_size > 1): 
            output_size = calcul_output(input_size, layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi, input_size
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(layer)
                return 1, archi, output_size
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

# Config Architecture

In [25]:
# choose the configuration of the architecture
def choose_config (iteration):
    esp = epsilon_value[random.randrange(5)]
    rate = dropout_value[random.randrange(7)]
    
    # chose the config
    if (iteration <=125):
        type_archi = TypeArchi(0) # config ALL
    elif(iteration <= 150):
        type_archi = TypeArchi(1) # config LeNet
    elif(iteration <= 175):
        type_archi = TypeArchi(2) # config ResNet
    else: type_archi = TypeArchi(3) # config DenseNet
    

    print(type_archi)

    ca = ConfArchi(type_archi=type_archi.value, 
                   epsilon=esp, 
                   dropout_rate=rate, 
                   compress_factor=0.5)
    return ca

In [26]:
def get_layer(ta):
    full_extraction_feature = [] # full feature extraction
    wp_extraction_feature = [] # feature extraction without pooling

    type_archi = TypeArchi(ta)
    # all layer
    if (type_archi == TypeArchi.ALL):
        full_extraction_feature= [Convolution, Pooling, IdBlock, ConvBlock, DenseNetBlock] 
        wp_extraction_feature= [Convolution, IdBlock, ConvBlock, DenseNetBlock]
        spe_block = False
    # if Lenet
    elif (type_archi == TypeArchi.LENET):
        full_extraction_feature = [Convolution, Pooling]
        wp_extraction_feature = [Convolution]
        spe_block = True
    # if resnet
    elif (type_archi == TypeArchi.RESNET):
        full_extraction_feature= [Convolution, Pooling, IdBlock, ConvBlock]
        wp_extraction_feature= [Convolution, IdBlock, ConvBlock]
        spe_block = False
    # if denset
    elif(type_archi == TypeArchi.DENSENET): 
        full_extraction_feature= [Convolution, Pooling, DenseNetBlock]
        wp_extraction_feature= [Convolution, DenseNetBlock]
        spe_block = False
    else:
        print("get_layer Error")
    return full_extraction_feature, wp_extraction_feature, spe_block

# Creation d'architecture

In [27]:
for i in range(101,201):
    ca = choose_config(i)
    full_extraction_feature, wp_extraction_feature, spe_block = get_layer(ca.type_archi)

    architecture = list()

    nb_filter_value = 3 # init nb feature map
    output_size = 32

    # add input Layer
    architecture.append(InputLayer(shape=[32,32,3]))

    # add extraction feature (succession of Pooling/convolution)
    # Pooling can't be follow by a Pooling
    pooling = True
    code = 1 # we can add new layer
    j = 6
    while( (not spe_block) | (output_size == 32) | ((stop(len(architecture),7)) & (code == 1))):
        # not get 2 pooling in a row
        if(pooling):
            layer = wp_extraction_feature[random.randrange(len(wp_extraction_feature))]
            pooling = False
        else:
            layer = full_extraction_feature[random.randrange(len(full_extraction_feature))]
            if(layer == Pooling):
                pooling = True

        # incrementation of number of filter
        if(layer == Convolution or layer == ConvBlock):
            nb_filter_value = nb_filter_value*j
            j=2

        # Kernel <= output/2
        if(output_size == 2):
            kernel_value_filtered = [1,2]
        elif(output_size == 1):
            kernel_value_filtered = [1]
        else: 
            kernel_value_filtered = [value for value in kernel_value if value <= math.ceil(output_size/2)]

        kernel=kernel_value_filtered[random.randrange(len(kernel_value_filtered))]

        # kernel >= stride
        kernel=kernel_value[random.randrange(6)]
        stride_value_filtered = [value for value in stride_value if value <= kernel]
        stride = stride_value_filtered[random.randrange(len(stride_value_filtered))]

        # if layer is Convolution
        if(layer == Convolution):
            add_layer = Convolution(
                kernel=kernel, 
                padding=padding_value[random.randrange(2)], 
                stride=stride, 
                nb_filter= nb_filter_value,
                fct_activation=fct_activation_value[random.randrange(3)]
            )
        # if is pooling
        elif(layer == Pooling):
            add_layer = Pooling(
                op = op_value[random.randrange(2)],
                kernel=kernel, 
                padding=padding_value[random.randrange(2)], 
                stride=stride
            )
        # if is identity block
        elif(layer == IdBlock):
            add_layer = IdBlock(
                kernel=kernel, 
                padding='same', 
                nb_filter=nb_filter_value, 
                fct_activation=fct_activation_value[random.randrange(3)])
            spe_block = True
            # if is convolution block
        elif (layer == ConvBlock):
            add_layer = ConvBlock(
                kernel=kernel, 
                nb_filter=nb_filter_value, 
                padding='same', 
                stride=stride,
                fct_activation=fct_activation_value[random.randrange(3)])
            spe_block = True
        elif (layer == DenseNetBlock):
            add_layer = DenseNetBlock( 
                kernel = kernel, 
                nb_filter = nb_filter_value, 
                padding='same', 
                nb_layer = nb_layer_value[random.randrange(3)],
                stride = stride,
                fct_activation=fct_activation_value[random.randrange(3)],
                op = op_value[random.randrange(2)],
                nb_block_densenet= nb_block_densenet_value[random.randrange(5)])
            spe_block = True
        else: 
            print("error")

        code, architecture, output_size = addLayer(architecture, add_layer)

    try:
        nb_filter_value = architecture[len(architecture)-1]['nb_filter']
    except:
        nb_filter_value = int(nb_filter_value/2)

    print("output size : " + str(output_size))
    print("nb_filter : " + str(nb_filter_value))
    print("code: " + str(code))


    clf_layer = classification[random.randrange(2)]
    print(clf_layer)
    if clf_layer == Flatten:
        # add flatten layer
        architecture.append(Flatten())


        # compute parameters
        param = output_size*output_size*(nb_filter_value)
        print('param : ' + str(param))


        ## init values
        nb=0
        if (param >= 1000):
            nb_neurones = int(param*20/100)
            print("=====")
            print("iteration" + str(nb))
            print("nb_neurone : " + str(nb_neurones))
            dense = Dense(
                nb_neurones = nb_neurones,
                fct_activation = fct_activation_value[random.randrange(3)]
            )
            architecture.append(dense)
            nb+=1
            nb_neurones = int(nb_neurones*20/100)

        else:
            pourcent = random.uniform(10,90)
            nb_neurones = int(param*pourcent/100)

        while(nb_neurones >= 1000):
            print("=====")
            print("iteration" + str(nb))
            print("nb_neurone : " + str(nb_neurones))
            dense = Dense(
                nb_neurones = nb_neurones,
                fct_activation = fct_activation_value[random.randrange(3)]
            )
            architecture.append(dense)
            nb+=1
            nb_neurones = int(nb_neurones*20/100)


        ## add dense Layers
        while(nb_neurones > nb_class*10):
            print("=====")
            print("iteration" + str(nb))
            print("nb_neurone : " + str(nb_neurones))

            dense = Dense(
                nb_neurones = nb_neurones,
                fct_activation = fct_activation_value[random.randrange(3)]
            )
            architecture.append(dense)


            #incrementation
            pourcent = random.uniform(10,90)
            nb+=1
            nb_neurones = int(nb_neurones*pourcent/100)

    elif (clf_layer == GlobalPooling):
        architecture.append(GlobalPooling(op=op_value[random.randrange(2)]))
    else: print(clf_layer == GlobalPooling)


    last_dense = Dense(
        nb_neurones = nb_class,
        fct_activation="softmax"           
    )

    architecture.append(last_dense)

    # ajout config
    architecture.append(ca)

    # print architecture
    print(architecture)

    # create file
    file_name = "archi_V3_%d.json" % i
    print(file_name)
    print("")
    print("")
    create_json_file(architecture, file_name)
        

TypeArchi.ALL
output size : 3
nb_filter : 36
code: -1
<class '__main__.Flatten'>
param : 324
=====
iteration0
nb_neurone : 275
[InputLayer(shape=[32, 32, 3]), ConvBlock(fct_activation='tanh', kernel=2, nb_filter=18, padding='same', stride=1), Convolution(fct_activation='tanh', kernel=7, nb_filter=36, padding='same', stride=7), DenseNetBlock(fct_activation='relu', kernel=4, nb_block_densenet=5, nb_filter=36, nb_layer=1, op='max', padding='same', stride=2), Flatten(), Dense(fct_activation='tanh', nb_neurones=275), Dense(fct_activation='softmax', nb_neurones=10), ConfArchi(compress_factor=0.5, dropout_rate=0.8, epsilon=1.001e-05, type_archi=0)]
archi_V3_101.json


TypeArchi.ALL
output size : 1
nb_filter : 72
code: 0
<class '__main__.Flatten'>
param : 72
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activation='selu', kernel=4, nb_filter=18, padding='valid', stride=2), ConvBlock(fct_activation='relu', kernel=5, nb_filter=36, padding='same', stride=5), Convolution(fct_activation='selu', k

TypeArchi.LENET
output size : 1
nb_filter : 288
code: 0
<class '__main__.Flatten'>
param : 288
=====
iteration0
nb_neurone : 146
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activation='relu', kernel=2, nb_filter=18, padding='same', stride=1), Convolution(fct_activation='tanh', kernel=4, nb_filter=36, padding='valid', stride=1), Convolution(fct_activation='relu', kernel=2, nb_filter=72, padding='valid', stride=2), Pooling(kernel=2, op='max', padding='valid', stride=2), Convolution(fct_activation='selu', kernel=6, nb_filter=144, padding='same', stride=6), Convolution(fct_activation='relu', kernel=6, nb_filter=288, padding='same', stride=3), Flatten(), Dense(fct_activation='tanh', nb_neurones=146), Dense(fct_activation='softmax', nb_neurones=10), ConfArchi(compress_factor=0.5, dropout_rate=0.8, epsilon=1.1e-07, type_archi=1)]
archi_V3_139.json


TypeArchi.LENET
output size : 1
nb_filter : 72
code: 0
<class '__main__.GlobalPooling'>
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activ

TypeArchi.DENSENET
output size : 1
nb_filter : 36
code: 0
<class '__main__.Flatten'>
param : 36
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activation='selu', kernel=6, nb_filter=18, padding='valid', stride=1), Convolution(fct_activation='relu', kernel=6, nb_filter=36, padding='same', stride=5), DenseNetBlock(fct_activation='selu', kernel=7, nb_block_densenet=4, nb_filter=36, nb_layer=3, op='max', padding='same', stride=7), Flatten(), Dense(fct_activation='softmax', nb_neurones=10), ConfArchi(compress_factor=0.5, dropout_rate=0.0, epsilon=1.1e-05, type_archi=3)]
archi_V3_177.json


TypeArchi.DENSENET
output size : 1
nb_filter : 9
code: 0
<class '__main__.Flatten'>
param : 9
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activation='relu', kernel=6, nb_filter=18, padding='same', stride=4), Pooling(kernel=5, op='max', padding='valid', stride=2), DenseNetBlock(fct_activation='relu', kernel=3, nb_block_densenet=4, nb_filter=18, nb_layer=2, op='avg', padding='same', stride=1), Pooling(